In [10]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator


In [11]:
filename = 'GOOG.csv'  # "data/solusdt_15m_data.csv"  # 
symbol = filename
df = pd.read_csv(filename)
df.info()

datetime_name = 'Date'  # 'open_datetime'  # 
price_name = 'Close'

df[datetime_name] = pd.to_datetime(df[datetime_name])   # , unit='ms')
df.corr()
#dr = df.set_axis(df[datetime_name], copy=False)
df = df[[datetime_name, price_name]]

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3543 entries, 0 to 3542
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3543 non-null   object 
 1   Open       3543 non-null   float64
 2   High       3543 non-null   float64
 3   Low        3543 non-null   float64
 4   Close      3543 non-null   float64
 5   Adj Close  3543 non-null   float64
 6   Volume     3543 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 193.9+ KB


,Date,Close
0,2005-08-19,6.973860
1,2005-08-22,6.824669
2,2005-08-23,6.963399
3,2005-08-24,7.037870
4,2005-08-25,7.038368


In [12]:
import plotly
import plotly.graph_objects as go
print("Plotly Version: ",plotly.__version__)


Plotly Version:  5.14.1


In [13]:
trace = go.Scatter(
    x = df[datetime_name],
    y = df[price_name],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = f'{symbol} Price',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close (Dollars)"}
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [14]:
close_data = df[price_name].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df[datetime_name][:split]
date_test = df[datetime_name][split:]

print(len(close_train))
print(len(close_test))


2834
709


In [15]:
look_back = 15


train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)


Epoch 1/50


/tmp/ipykernel_2208630/1344676287.py:20: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



141/141 [==============================] - 2s 5ms/step - loss: 227.5367
Epoch 2/50
141/141 [==============================] - 1s 5ms/step - loss: 4.5218
Epoch 3/50
141/141 [==============================] - 1s 5ms/step - loss: 3.0678
Epoch 4/50
141/141 [==============================] - 1s 5ms/step - loss: 0.8716
Epoch 5/50
141/141 [==============================] - 1s 5ms/step - loss: 0.5868
Epoch 6/50
141/141 [==============================] - 1s 5ms/step - loss: 0.4553
Epoch 7/50
141/141 [==============================] - 1s 5ms/step - loss: 0.3974
Epoch 8/50
141/141 [==============================] - 1s 5ms/step - loss: 0.3668
Epoch 9/50
141/141 [==============================] - 1s 5ms/step - loss: 0.3656
Epoch 10/50
141/141 [==============================] - 1s 5ms/step - loss: 0.3903
Epoch 11/50
141/141 [==============================] - 1s 5ms/step - loss: 0.3507
Epoch 12/50
141/141 [==============================] - 1s 5ms/step - loss: 0.3704
Epoch 13/50
141/141 [=============

In [ ]:
prediction = model.predict_generator(test_generator)
# prediction


/tmp/ipykernel_2208630/1599169009.py:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = f'{symbol} Price',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()


In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    print(num_prediction)
    last_date = df[datetime_name].values[-1]
    print(last_date)
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1,  freq='D').tolist()
    print(prediction_dates)
    return prediction_dates

num_prediction = 4
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

trace1 = go.Scatter(
    x = df[datetime_name].tolist(),
    y = close_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = f'{symbol} Price',
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

1/1 [==============================] - 0s 18ms/step
4
2019-09-17T00:00:00.000000000
[Timestamp('2019-09-17 00:00:00'), Timestamp('2019-09-17 00:15:00'), Timestamp('2019-09-17 00:30:00'), Timestamp('2019-09-17 00:45:00'), Timestamp('2019-09-17 01:00:00')]
